In [1]:
import pandas as pd
import os
from openai import OpenAI
from dotenv import load_dotenv

from crewai import Agent, Task, Crew, Process
# from crewai_tools import SerperDevTool, FileReadTool, ScrapeWebsiteTool, WebsiteSearchTool
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\limya\AppData\Local\sagemaker\sagemaker\config.yaml


In [2]:
load_dotenv('.env')
os.environ['OPENAI_MODEL_NAME'] = "gpt-4o-mini"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [3]:
topic = input("Enter the audit topic: ")
if topic:
    print("You entered:", topic)
else:
    print("Please enter an audit task.")

You entered: baby bonus


In [4]:
tool_search = SerperDevTool(n_results=3, country="sg")
tool_webscrape = ScrapeWebsiteTool()

auditor = Agent(
    role = "auditor",
    goal="Return the Audit Planning Memo on {topic} based on inputs from...",
    backstory = """You are an auditor. You need to consider the inputs from the researcher to determine what to write for the audit planning memo and risk assessment""",
    allow_delegation = True,
    verbose = True
)

researcher = Agent(
    role="researcher",
    goal="From the internet, research and analyse and sieve out information related to {topic} only." ,
    backstory="""As a researcher, navigating and extracting critical information is crucial. You are assisting the auditor to research on topics relevant to {topic}. 
    """,
    allow_delegation=False,
    verbose=True,
)


task_search = Task(
    description="""Returns a JSON list of websites that contains the top 5 most relevant information, in the context of Singapore, that is related to policy and regulation on {topic} only. Ignore urls that ends with .pdf.
    """,
    expected_output="""\
    Returns websites that contain the top 5 most relavant policy and regulation information on {topic} only. The output must be in JSON format with the description as the key and url as value.""",

    agent=researcher,
    tools=[tool_search],
)


task_websitesearch = Task(
    description="""Scrape through a website for information related to {topic} only
    """,
    expected_output="""\
    Summarise the website for information related to {topic} only in the Singapore context. """,

    agent=researcher,
    tools=[tool_webscrape],
    context = [task_search]
)

task_write= Task(
    description="""\
    1. Use the content to craft an audit planning memo and risk assessment based on {topic} .
    2. Sections are properly named in an engaging manner.
    3. Proofread for grammatical errors and alignment with the common style used in audit planning memos.""",

    expected_output="""
    A well-written audit planning memo and risk assessment in markdown format. Each section should have minimum of 2 and maximum of 5 paragraphs.""",
    agent=auditor,
    context=[task_websitesearch],
    output_file="APM.md"
)




In [5]:
#creating the crew

crew = Crew(
    agents=[auditor,researcher],
    tasks=[task_search,task_websitesearch,task_write],
    verbose=True
)


In [6]:
result = crew.kickoff(inputs={"topic": topic})

 [2024-10-08 09:26:19][DEBUG]: == Working Agent: researcher
 [2024-10-08 09:26:19][INFO]: == Starting Task: Returns a JSON list of websites that contains the top 5 most relevant information, in the context of Singapore, that is related to policy and regulation on baby bonus only. Ignore urls that ends with .pdf.
    


> Entering new CrewAgentExecutor chain...
Action: Search the internet  
Action Input: {"search_query": "Singapore baby bonus policy and regulation"}   


Search results: Title: Baby Bonus Scheme - Made For Families
Link: https://www.madeforfamilies.gov.sg/support-measures/raising-your-child/financial-support/baby-bonus-scheme
Snippet: Your child must obtain Singapore Citizenship before 24 months of age to be eligible for the Cash Gift, and before 12 years of age to be eligible for the Child ...
---
Title: [PDF] terms and conditions for the baby bonus scheme
Link: https://www.babybonus.msf.gov.sg/Documents/Terms%20and%20Conditions%20for%20Baby%20Bonus%20Scheme%20%2828%20J

In [7]:
print(f"Raw Output: {result.raw}")
print(f"Token Usage: {result.token_usage}")
print("-----------------------------------------\n\n")
print(f"Tasks Output of Task 1: {result.tasks_output[0]}")
print("-----------------------------------------\n\n")

Raw Output: ```markdown
# Audit Planning Memo: Baby Bonus Scheme

## Introduction
This audit planning memo outlines the key components, risks, and statistics associated with the Baby Bonus Scheme in Singapore, a government initiative designed to assist families with the financial burdens of raising children. The objective of this audit is to evaluate the effectiveness of the scheme, identify potential risks, and ensure compliance with established guidelines. The Baby Bonus Scheme is part of the broader Marriage and Parenthood Package, aiming to encourage higher birth rates in Singapore.

## Overview of the Baby Bonus Scheme
The Baby Bonus Scheme comprises two primary components: the Baby Bonus Cash Gift (BBCG) and the Child Development Account (CDA). The BBCG provides direct financial assistance to eligible parents, varying based on the birth order of the child. For instance, the first child attracts up to $11,000, while subsequent children receive varying amounts, with the third child

In [ ]:
# tool = SerperDevTool(n_results=3, country="sg")

# agent = Agent(
#     role="search agent",
#     goal="return most relevant information, in the context of Singapore, that is related to policy and regulation on {topic}" ,
#     backstory="""You're working on gathering all relavant policy and regualtion information on {topic}.
#     """,
#     allow_delegation=False,
#     verbose=True,
#     max_iter=40
# )

# task = Task(
#     description="""return most relevant information, in the context of Singapore, that is related to policy and regulation on {topic}
#     """,
#     expected_output="""\
#     A detailed report with most relevant information information that is related to policy and regulation on {topic}.""",

#     agent=agent,
#     tools=[tool],
# )

# crew = Crew(
#     agents=[agent],
#     tasks=[task],
#     verbose=True
# )

# result = crew.kickoff(inputs={"topic": topic})

In [ ]:
# print(f"Raw Output: {result.raw}")
# print(f"Token Usage: {result.token_usage}")
# print("-----------------------------------------\n\n")
# print(f"Tasks Output of Task 1: {result.tasks_output[0]}")
# print("-----------------------------------------\n\n")
# # print(f"Tasks Output of Task 2: {result.tasks_output[1]}")